In [1]:
# Standard library imports
import multiprocessing
import warnings

# Third party imports
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from scipy.stats import norm, randint, uniform
from sklearn import metrics
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, confusion_matrix)
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from lightgbm import LGBMRegressor
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Local application/library specific imports
from Preprocessing_singlebuild_with_artificial import Preprocessing
from Visualization_artificial import Visualization
import data_wrangling

# Supress warnings and pandas chained assignment
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'



<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Reference method based on Thresholding method based on non-parametric dynamic thresholding method from Zhang et al., “RobustProphet.” with LightGBM for the article: 
## Building consumption anomaly detection: A comparative study of two probabilistic approaches 
Davor Stjelja, Vladimir Kuzmanovski, Risto Kosonen, Juha Jokisalo 


Aalto University

Granlund Oy

Vaisala Oyj

Nanjing Tech University


### Only data from buildings from BDG2 dataset are available for test

In [2]:
from common import add_noise, AnomalyConfig, ColumnConfig
config_heating = {
    'anomaly_periods': {
        'Hog_office_Marlena': ['2017-10-25', '2017-12-24'],
        'Hog_office_Bessie': ['2017-10-25', '2017-12-24']},
    'stabilization': "None",
    'lgbm_params': {
        'learning_rate': 0.037987557990657006,
        'max_depth': 27,
        'min_data_in_leaf': 527,
        'n_estimators': 783,
        'num_leaves': 1786,
        'objective': 'regression',
        'random_state': 59,
        'reg_alpha': 0.16284813849651236,
        'reg_lambda': 0.4198737911470489},
    'length':750,
    'xgblss_params':{
            'eta': 0.4195606966617469,
            'max_depth': 2,
            'gamma': 0.0003978587346077087,
            'subsample': 0.8450425561621635,
            'colsample_bytree': 0.5172456512864724,
            'min_child_weight': 0.006595871665786983,
            'booster': 'gbtree',
            'opt_rounds': 498
        },
}

config_electricity = {
    'anomaly_periods' : {
        'Hog_education_Bruno': ['2017-09-21', '2017-11-20'],
        'Hog_office_Corie': ['2017-09-21', '2017-11-20']},
    'stabilization': "MAD",
    'lgbm_params': {
        'learning_rate': 0.037987557990657006,
        'max_depth': 27,
        'min_data_in_leaf': 527,
        'n_estimators': 783,
        'num_leaves': 1786,
        'objective': 'regression',
        'random_state': 59,
        'reg_alpha': 0.16284813849651236,
        'reg_lambda': 0.4198737911470489},
    'length':3500,
    'xgblss_params':{'eta': 0.19262478591119223,
        'max_depth': 8,
        'gamma': 0.12743803499652331,
        'subsample': 0.9234424599017936,
        'colsample_bytree': 0.8222027451145232,
        'min_child_weight': 0.0001523605987501173,
        'booster': 'gbtree',
        'opt_rounds': 312}, 
}


def get_noise_config(noise_level):
    config_noise_low = {
        "noise_level": 0.1,
        "num_events": 72,
        "event_window_in_hours": 3
    }

    config_noise_medium = {
        "noise_level": 0.5,
        "num_events": 110,
        "event_window_in_hours": 3
    }

    config_noise_high = {
        "noise_level": 0.75,
        "num_events": 250,
        "event_window_in_hours": 3
    }

    if noise_level == "Low":
        return config_noise_low
    elif noise_level == "Medium":
        return config_noise_medium
    elif noise_level == "High":
        return config_noise_high
    else:  # None or any other value
        return None

# General hyperparams
B = 3                       # number of ensembles
alpha = 0.05         # confidence level
gap = 24            # used for MAPIE
quantiles = [alpha/2,       # quantiles to predict
             0.5,
             1-(alpha/2)]
time_steps_out=24


# Store the configuration in a dictionary
P = {'B':B, 'alpha':alpha, 'quantiles':quantiles,'time_steps_out':time_steps_out}

# User Selection

In [3]:
# Uncomment to load dataset
dataset_filename = 'Public_Heat_dataset.csv'  # For heating
#dataset_filename = 'Public_Elec_dataset.csv'  # For electricity

# User selects the noise level
user_selected_noise_level = "None"  # Can be None, Low, Medium, or High

In [4]:
ds = pd.read_csv(dataset_filename)
ds['Date']=pd.to_datetime(ds['Date'])
# Get the corresponding noise configuration
selected_noise_config = get_noise_config(user_selected_noise_level)
# Assume user_selected_noise_level is defined as before
noise_level = user_selected_noise_level if user_selected_noise_level else "NoNoise"
# Determine the type of dataset from the filename
if 'Heat' in dataset_filename:
    current_config = config_heating
    dataset_type = 'Heat'
else:
    current_config = config_electricity
    dataset_type = 'Elec'

ds.drop_duplicates(subset=['ObjectName','Date'],inplace=True)
anomaly_periods = current_config['anomaly_periods']

### Aritifical anomalies

In [6]:
# Uncomment next two lines if you want to check anomalies
ds.drop(['Consumption'], axis=1, inplace=True)
ds.rename(columns={'m_Consumption':'Consumption'}, inplace=True)

# Uncomment next two lines if you want to check clean data
# ds.drop(['m_Consumption'], axis=1, inplace=True)
# ds['m_bool'] = False

### Data pre-processing

In [7]:
ds['Temperature'] = ds['Temperature'].interpolate(method='linear')
ds['Consumption'] = ds['Consumption'].interpolate(method='linear')
# Use the method with inline=True if the original dataset can be updated, otherwise use inplace=False and write the output in a new variable

# Add Temperature value in each row from 1,2,3,6,12,24,48 and 72 hours ago
data_wrangling.add_lagged_values(ds, params={ 'group_by': ['ObjectName'], 'arrange_by': ['ObjectName','Date'], 'value_col': 'Temperature', 'shift': [1,2,3,6,12,24,48,72], 'col_namespace': 'T_lag_%sh'}, inplace = True)

# Add Temperature average values in each row from 1,2,3,6,12,24,48 and 72 hours ago
data_wrangling.add_lagged_means(ds, params={ 'group_by': ['ObjectName'], 'arrange_by': ['ObjectName','Date'], 'value_col': 'Temperature', 'shift': [2,3,6,12,24,48,72], 'col_namespace': 'T_avg_%sh'}, inplace = True)

ds['Month']=ds['Date'].dt.month
ds['Hour']=ds['Date'].dt.hour
ds['weekday']=ds['Date'].dt.day_of_week

### Create Noise

In [8]:
if selected_noise_config is not None:
    # If noise configuration is provided, apply noise
    ds_noise = pd.DataFrame()
    for building_id, period in anomaly_periods.items():
        Start_date = pd.Timestamp(period[0])
        End_date = pd.Timestamp(period[1])
        Test_Start = Start_date
        Train_End = Test_Start + pd.DateOffset(days=-1)
        Train_Start = Test_Start + pd.DateOffset(months=-12)
        Test_End = End_date

        ds_build = ds[ds.ObjectName == building_id]

        config = AnomalyConfig(
            column=ColumnConfig(
                time="Date",
                target="Consumption",
            ),
            start_date=Train_Start,
            end_date=Train_End,
            **selected_noise_config )

        ds_build["m_bool_original"] = ds_build["m_bool"].copy()
        add_noise(ds_build, config, inplace=True)
        ds_build["m_bool"] = ds_build[["m_bool", "m_bool_original"]].any(axis=1)
        
        # Drop the temporary and unnecessary columns
        ds_build.drop(["m_bool_original", "Consumption"], axis=1, inplace=True)
        
        # Rename the modified column back to its original name
        ds_build.rename(columns={'m_Consumption': 'Consumption'}, inplace=True)

        # Append the processed building data to the main DataFrame
        ds_noise = pd.concat([ds_noise,ds_build])
    # Reset the index of the final DataFrame
    ds_noise.reset_index(drop=True, inplace=True)
    ds=ds_noise
else:
    # Skip noise creation
    pass


## Reference method

In [9]:
lgbm_params = current_config['lgbm_params']
model = LGBMRegressor(**lgbm_params)


#### Train

In [10]:
Results_LGBM=pd.DataFrame()
for building_id, period in anomaly_periods.items():
    Start_date=pd.Timestamp(period[0])
    End_date=pd.Timestamp(period[1])

    Test_Start=Start_date
    Train_End=Test_Start+DateOffset(days=-1)
    Train_Start=Test_Start+DateOffset(months=-12)
    Test_End=End_date
    Results_temp=pd.DataFrame()
    ds_build=ds[ds.ObjectName == building_id]
    X_train,X_test,Y_train,Y_test,Results_temp=Preprocessing(ds_build,Train_Start,Train_End,Test_Start,Test_End)
    if X_train.empty or X_test.empty or Y_train.empty or Y_test.empty:
        continue
    num_decimal_points = -int(np.log10(Y_train.target.describe()['std']))
    if num_decimal_points > 1:
        Y_train *= 100
        Y_test *= 100
        Results_temp[['Consumption']] *= 100
    # MAPIE LGBM
    y_train=Y_train['target']
    y_test=Y_test['Consumption']

    model = model.fit(X_train, y_train)

    y_pred = model.predict(
        X_test, alpha=alpha, ensemble=True, optimize_beta=True
    )
    y_pred=np.expm1(y_pred)

    Results_temp['Predicted']=pd.Series(y_pred)


    Results_LGBM= pd.concat([Results_LGBM, Results_temp], axis=0)


[LightGBM] [Warning] min_data_in_leaf is set=527, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=527
[LightGBM] [Warning] min_data_in_leaf is set=527, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=527


##  Results

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

def alarm_day_performance(df1):
    # Freshness check checkes was there any hour in a day which had alarm in df1 but not in df2, then it outputs df2 with column with such days marked
    # Idea is to visualize freshly learned drift, which has not yet been learned by long model, but did with short model

    df1['date'] = df1['Date'].dt.date


    days = df1.loc[df1['IsAnomaly'].eq(1), 'date']
    df1['alarm_thatday'] = df1['date'].isin(days).astype(int)
    days_anomal=df1.loc[(df1['m_bool']).eq(1), 'date']
    df1['anomal_thatday'] = df1['date'].isin(days_anomal).astype(int)
    return df1


def evaluate_model_performance(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)

    metrics_df = pd.DataFrame({'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score', 'ROC-AUC'],
                               'Value': [accuracy, precision, recall, f1, roc_auc]}).set_index('Metric')

    return metrics_df, conf_matrix

### Thresholding method based on non-parametric dynamic thresholding method from Zhang et al., “RobustProphet.”

In [12]:
def calculate_hourly_thresholds(df, z_values, alpha=0.3):
    # Convert 'Date' to datetime if it's not already
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Initialize a list to store the results
    results_list = []

    for object_name in df['ObjectName'].unique():
        object_data = df[df['ObjectName'] == object_name].sort_values(by='Date')

        for index, row in object_data.iterrows():
            # Data up to the current hour
            historical_data = object_data[object_data['Date'] <= row['Date']]

            y_true = historical_data['Consumption'].values
            y_pred = historical_data['Predicted'].values

            # Calculate prediction errors
            errors = np.abs(y_true - y_pred)

            # Apply EWMA to smooth errors
            errors_smoothed = pd.Series(errors).ewm(alpha=alpha).mean().values

            # Find the best threshold for the current hour
            best_threshold, best_score = float('-inf'), float('-inf')
            for z in z_values:
                threshold = np.mean(errors_smoothed) + z * np.std(errors_smoothed)
                outliers = errors_smoothed > threshold
                delta_mu = np.mean(errors_smoothed) - np.mean(errors_smoothed[~outliers])
                delta_sigma = np.std(errors_smoothed) - np.std(errors_smoothed[~outliers])
                num_outliers = np.sum(outliers)
                num_continuous_sequences = np.sum(np.diff(np.where(outliers)[0]) != 1)
                score = (delta_mu + delta_sigma) / (num_outliers + num_continuous_sequences)
                if score > best_score:
                    best_threshold, best_score = threshold, score

            # Check if the current hour is an outlier
            is_anomaly = errors_smoothed[-1] > best_threshold

            # Append results for the current hour to the list
            results_list.append({
                'ObjectName': object_name,
                'Date': row['Date'],
                'Threshold': best_threshold,
                'IsAnomaly': is_anomaly
            })

    # Create a DataFrame from the results list
    results = pd.DataFrame(results_list)
    return results

# Example usage

z_values = np.linspace(1, 3, 10)  # Range of z-values to explore

hourly_thresholds_and_outliers = calculate_hourly_thresholds(Results_LGBM, z_values)
# Merge the two dataframes on 'ObjectName' and 'Date'
Results_LGBM = pd.merge(Results_LGBM, hourly_thresholds_and_outliers, on=['ObjectName', 'Date'])
# Filter out rows where 'Threshold' is -inf
Results_LGBM = Results_LGBM[Results_LGBM['Threshold'] != -np.inf] 


C:\Users\DSt\AppData\Local\Temp\ipykernel_32232\2978696753.py:33: RuntimeWarning: invalid value encountered in scalar divide
  score = (delta_mu + delta_sigma) / (num_outliers + num_continuous_sequences)
C:\Users\DSt\AppData\Local\Temp\ipykernel_32232\2978696753.py:33: RuntimeWarning: invalid value encountered in scalar divide
  score = (delta_mu + delta_sigma) / (num_outliers + num_continuous_sequences)


In [19]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn import metrics


listofBuildings=Results_LGBM['ObjectName'].unique()
fig = make_subplots(rows=len(listofBuildings), cols=1,shared_xaxes=False)

CV_RMSE=[]
EV=[]
RMSE=[]
RRSE=[]
i=1
alarm_cols = ['IsAnomaly'] # [ alarm_flag ]
alarm_colors = ['orange']
for unique in listofBuildings:
    tempdf=Results_LGBM[Results_LGBM['ObjectName']==unique]
    tempdf=tempdf.groupby(['Date'])[['Consumption','Predicted','m_bool']+ alarm_cols].first()
    rmse = round(np.sqrt(metrics.mean_squared_error(tempdf['Consumption'], tempdf['Predicted'])),4)
    cv_rmse=round(rmse/np.mean(tempdf['Consumption'])*100,2)
    ev=round(metrics.explained_variance_score(tempdf['Consumption'], tempdf['Predicted']),4)*100
    rrse = round(np.sqrt( np.sum((tempdf['Consumption'] - tempdf['Predicted'])**2, axis=0)/np.sum((tempdf['Consumption'] - np.mean(tempdf['Consumption']))**2, axis=0))*100,2)

    CV_RMSE.append(cv_rmse)
    EV.append(ev)
    RRSE.append(rrse)
    RMSE.append(rmse)

    fig.add_trace(go.Scatter(x=tempdf.index,y=tempdf['Consumption'],name=('Measured '),line=dict(color="gray")),row=i,col=1)
    fig.add_trace(go.Scatter(x=tempdf.index,y=tempdf['Predicted'],name=('Predicted '),line=dict(color='red') ),row=i,col=1)
    #fig.add_trace(go.Scatter(x=tempdf.index,y=tempdf['Upper bound'] ,name=('Upper bound '), mode='lines', marker=dict(color='rgba(255, 0, 0, 0.1)'), line=dict(width=0)),row=i,col=1)
    #fig.add_trace(go.Scatter(x=tempdf.index,y=tempdf['Lower bound'] ,name=('Lower bound '), mode='lines', marker=dict(color='rgba(255, 0, 0, 0.1)'), line=dict(width=0), fillcolor='rgba(255, 0, 0, 0.1)', fill='tonexty'),row=i,col=1)
    for alrm in range(len(alarm_cols)):
        alarmdf=tempdf.loc[tempdf[alarm_cols[alrm]] == True]
        fig.add_trace(go.Scatter(x=alarmdf.index,y=alarmdf['Consumption'],name=('Alarm [' + str(alrm + 1) + '] ' + unique), mode='markers', marker=dict(color=alarm_colors[alrm],opacity=0.5, size = 12 - (alrm * 3)) ),row=i,col=1)

    artificial_anom=tempdf.loc[tempdf['m_bool'] == True]
    fig.add_trace(go.Scatter(x=artificial_anom.index,y=artificial_anom['Consumption'], mode='markers', marker=dict(color='black', size = 6) ),row=i,col=1)
    fig.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.1, showarrow=False,
                text=f'{unique} CV_RMSE: {cv_rmse}, RMSE: {rmse}, RRSE: {rrse}', row=i, col=1)
    i=i+1
fig.update_layout(showlegend=False, template='plotly_white', title='LGBM',height=750)
fig.show()

In [20]:
def alarm_results(df):
    # Initialize an empty dictionary to hold the metrics for each ObjectName
    metrics_dict = {}
    # Loop over each unique ObjectName
    for object_name in df['ObjectName'].unique():
        # Subset the dataframe to only rows for this ObjectName
        subset = df[df['ObjectName'] == object_name]
        #Alarms_back=alarm_matrix_new_withbackprop.Alarm_matrix(subset)
        matrix=alarm_day_performance(subset)
        metrix_df,conf_matrix=evaluate_model_performance(matrix['anomal_thatday'], matrix['alarm_thatday'])

        # Instead of concatenating, save the dataframe in a dictionary with the key as object_name
        metrics_dict[object_name] = metrix_df

    # After the loop, concatenate the dictionary into a dataframe, setting keys as the column names
    metrics_df = pd.concat(metrics_dict, axis=1).round(2)
    return metrics_df

In [21]:
alarm_results(Results_LGBM)

,Hog_office_Marlena,Hog_office_Bessie
,Value,Value
Metric,,
Accuracy,0.92,0.87
Precision,0.89,0.84
Recall,0.67,0.62
F1-score,0.76,0.72
ROC-AUC,0.82,0.79
